In [1]:
! pip install --user kubernetes

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from pprint import pprint
from kubernetes import client, config


In [34]:
pytorch_job = {
    "apiVersion": "kubeflow.org/v1",
    "kind": "PyTorchJob",
    "metadata": {
        "name": "pytorch-dist-ranzcr-3",
        "namespace": "kubeflow-user",
    },
    "spec": {
        "pytorchReplicaSpecs": {
            "Master": {
                "replicas": 1,
                "restartPolicy": "OnFailure",
                "template": {
                    "metadata": {
                        "annotations": {
                            "sidecar.istio.io/inject": "false"
                        }
                    },
                    "spec": {
                        "containers": [
                            {
                                "args": [
                                    "--epochs",
                                    "3",
                                    "--batch-size",
                                    "512",
                                    "--weight-decay",
                                    "0.1",
                                    "--image-size",
                                    "224",
                                    "--backend",
                                    "nccl"
                                ],
                                "image": "gcr.io/arrikto-playground/dimpo/ranzcr-dist:latest",
                                "name": "pytorch"
                            }
                        ]
                    }
                }
            },
            "Worker": {
                "replicas": 1,
                "restartPolicy": "OnFailure",
                "template": {
                    "metadata": {
                        "annotations": {
                            "sidecar.istio.io/inject": "false"
                        }
                    },
                    "spec": {
                        "containers": [
                            {
                                "args": [
                                    "--epochs",
                                    "3",
                                    "--batch-size",
                                    "512",
                                    "--weight-decay",
                                    "0.1",
                                    "--image-size",
                                    "224",
                                    "--backend",
                                    "nccl"
                                ],
                                "image": "gcr.io/arrikto-playground/dimpo/ranzcr-dist:latest",
                                "name": "pytorch",
                                "resources": {
                                    "limits": {
                                        "nvidia.com/gpu": 1
                                    }
                                }
                            }
                        ]
                    }
                }
            }
        }
    }
}

config.load_incluster_config()
api = client.CustomObjectsApi()
api.create_namespaced_custom_object(
    group="kubeflow.org",
    version="v1",
    namespace="kubeflow-user",
    plural="pytorchjobs",
    body=pytorch_job,
)
print("Resource created")


ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'Date': 'Thu, 18 Feb 2021 22:53:29 GMT', 'Content-Length': '129'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"Unauthorized","reason":"Unauthorized","code":401}

